<a href="https://colab.research.google.com/github/gracekfisher/bird-classifier/blob/main/Kaggle100Birds3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q tfds-nightly tensorflow matplotlib

     |████████████████████████████████| 4.3 MB 8.6 MB/s 
     |████████████████████████████████| 87 kB 6.5 MB/s 
     |████████████████████████████████| 462 kB 44.6 MB/s 


In [ ]:
! pip install kaggle

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download gpiosenka/100-bird-species

 99% 1.48G/1.49G [00:13<00:00, 149MB/s]
100% 1.49G/1.49G [00:13<00:00, 118MB/s]


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import pandas as pd
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, Conv1D, MaxPooling1D, Reshape
from tensorflow.keras.initializers import HeNormal
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical

In [ ]:
nb_classes=34

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten

# 1st Conv2D: 6 filters, 5x5
# 2nd Conv2D: 16 filters, 5x5

model = Sequential()
model.add(Conv2D(6, (5,5), activation='relu', kernel_initializer=HeNormal(), input_shape=(299,299,3,)))
model.add(MaxPool2D()) # filter 2x2, stride 2x2
model.add(Conv2D(16, (5,5), activation='relu', kernel_initializer=HeNormal()))
model.add(MaxPool2D()) # filter 2x2, stride 2x2
model.add(Conv2D(10, (5,5), activation='relu', kernel_initializer=HeNormal()))
model.add(Flatten())
model.add(Dense(nb_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   dtype = "float32",
                                   rotation_range=10,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

In [ ]:
valid_datagen = ImageDataGenerator(rescale=1./255,
                                   dtype="float32",
                                  )

In [ ]:
train_generator = train_datagen.flow_from_directory(
  'gdrive/MyDrive/Birds/train_cut', target_size=(299,299), 
  batch_size=20,
  class_mode='categorical')

Found 6668 images belonging to 34 classes.


In [ ]:
validation_generator = valid_datagen.flow_from_directory(
    'gdrive/MyDrive/Birds/test_cut', target_size=(299,299),
     batch_size=8, class_mode='categorical')

Found 170 images belonging to 34 classes.


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience=3, restore_best_weights=True, monitor='val_accuracy')

In [ ]:
model.fit(train_generator,
          steps_per_epoch=200, 
          validation_data=validation_generator,
          validation_steps=20, 
          epochs=15,
          callbacks=[es])

Epoch 1/15
200/200 [==============================] - 1103s 6s/step - loss: 3.5280 - accuracy: 0.0887 - val_loss: 3.0536 - val_accuracy: 0.2062
Epoch 2/15
200/200 [==============================] - 577s 3s/step - loss: 3.1218 - accuracy: 0.1490 - val_loss: 2.7209 - val_accuracy: 0.2375
Epoch 3/15
200/200 [==============================] - 420s 2s/step - loss: 2.9155 - accuracy: 0.2044 - val_loss: 2.3656 - val_accuracy: 0.3125
Epoch 4/15
200/200 [==============================] - 389s 2s/step - loss: 2.6732 - accuracy: 0.2681 - val_loss: 2.0273 - val_accuracy: 0.4250
Epoch 5/15
200/200 [==============================] - 387s 2s/step - loss: 2.4732 - accuracy: 0.3088 - val_loss: 1.6705 - val_accuracy: 0.5312
Epoch 6/15
200/200 [==============================] - 385s 2s/step - loss: 2.3056 - accuracy: 0.3475 - val_loss: 1.5258 - val_accuracy: 0.5875
Epoch 7/15
200/200 [==============================] - 387s 2s/step - loss: 2.2378 - accuracy: 0.3760 - val_loss: 1.6299 - val_accuracy: 0.543

In [ ]:
model.evaluate(validation_generator)

## **TOP 5**

In [ ]:
nb_classes=5

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, MaxPooling2D

# 1st Conv2D: 6 filters, 5x5
# 2nd Conv2D: 16 filters, 5x5

model = Sequential()
model.add(Conv2D(8, (7,7), activation='relu', kernel_initializer=HeNormal(), input_shape=(299,299,3)))
model.add(MaxPool2D()) # filter 2x2, stride 2x2
model.add(Conv2D(20, (7,7), activation='relu', kernel_initializer=HeNormal()))
model.add(Flatten())
model.add(Dense(nb_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
train_generator = train_datagen.flow_from_directory(
  'gdrive/MyDrive/Birds/train_top5', target_size=(299,299), 
  batch_size=20,
  class_mode='categorical')

Found 1146 images belonging to 5 classes.


In [ ]:
validation_generator = valid_datagen.flow_from_directory(
    'gdrive/MyDrive/Birds/test_top5', target_size=(299,299),
     batch_size=1, class_mode='categorical')

Found 25 images belonging to 5 classes.


In [ ]:
model.fit(train_generator,
          steps_per_epoch=100, 
          validation_data=validation_generator,
          validation_steps=20, 
          epochs=8,
          callbacks=[es])

Epoch 1/8
 20/100 [=====>........................] - ETA: 2:15 - loss: 1.4606 - accuracy: 0.2775

KeyboardInterrupt: ignored